# GPU架构基础 - 实践篇

本notebook通过实际代码帮助你理解GPU架构的核心概念。

**学习目标：**
- 使用CUDA API查询GPU硬件信息
- 理解SM、Warp、Thread的实际数值
- 观察Warp内线程的行为特性


## 环境准备

首先加载nvcc4jupyter扩展，使我们能够在Jupyter中编写CUDA C++代码。


In [ ]:
# 加载nvcc4jupyter扩展
%load_ext nvcc4jupyter


## 1. 查询GPU设备信息

使用 `cudaGetDeviceProperties` API 获取GPU的详细硬件信息。


In [ ]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>

int main() {
    int deviceCount;
    cudaGetDeviceCount(&deviceCount);
    
    printf("==========================================\n");
    printf("         GPU 设备信息查询\n");
    printf("==========================================\n\n");
    printf("检测到 %d 个CUDA设备\n\n", deviceCount);
    
    for (int i = 0; i < deviceCount; i++) {
        cudaDeviceProp prop;
        cudaGetDeviceProperties(&prop, i);
        
        printf("设备 %d: %s\n", i, prop.name);
        printf("------------------------------------------\n");
        
        // 计算能力
        printf("计算能力: %d.%d\n", prop.major, prop.minor);
        
        // SM相关
        printf("\n【SM信息】\n");
        printf("  SM数量: %d\n", prop.multiProcessorCount);
        printf("  每SM最大线程数: %d\n", prop.maxThreadsPerMultiProcessor);
        printf("  每SM最大Block数: %d\n", prop.maxBlocksPerMultiProcessor);
        
        // Warp相关
        printf("\n【Warp信息】\n");
        printf("  Warp大小: %d 线程\n", prop.warpSize);
        int maxWarpsPerSM = prop.maxThreadsPerMultiProcessor / prop.warpSize;
        printf("  每SM最大Warp数: %d\n", maxWarpsPerSM);
        
        // Block/Thread相关
        printf("\n【Block/Thread限制】\n");
        printf("  每Block最大线程数: %d\n", prop.maxThreadsPerBlock);
        printf("  Block维度限制: (%d, %d, %d)\n", 
               prop.maxThreadsDim[0], prop.maxThreadsDim[1], prop.maxThreadsDim[2]);
        printf("  Grid维度限制: (%d, %d, %d)\n", 
               prop.maxGridSize[0], prop.maxGridSize[1], prop.maxGridSize[2]);
        
        // 内存相关
        printf("\n【内存信息】\n");
        printf("  全局内存: %.2f GB\n", prop.totalGlobalMem / (1024.0 * 1024.0 * 1024.0));
        printf("  每Block共享内存: %zu KB\n", prop.sharedMemPerBlock / 1024);
        printf("  每SM共享内存: %zu KB\n", prop.sharedMemPerMultiprocessor / 1024);
        printf("  每Block寄存器数: %d\n", prop.regsPerBlock);
        printf("  每SM寄存器数: %d\n", prop.regsPerMultiprocessor);
        
        // 性能特性
        printf("\n【性能特性】\n");
        printf("  GPU时钟频率: %.2f GHz\n", prop.clockRate / 1e6);
        printf("  内存时钟频率: %.2f GHz\n", prop.memoryClockRate / 1e6);
        printf("  内存总线宽度: %d bit\n", prop.memoryBusWidth);
        printf("  L2缓存大小: %d KB\n", prop.l2CacheSize / 1024);
        
        printf("\n");
    }
    
    return 0;
}


## 2. 观察Warp内线程行为

Warp内的32个线程是同步执行的。让我们通过Warp级别的投票函数来验证这一点。


In [ ]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>

// 使用Warp级别的投票函数
__global__ void warpVoteDemo() {
    int tid = threadIdx.x;
    int warpId = tid / 32;
    int laneId = tid % 32;  // 线程在Warp内的位置
    
    // 条件：线程ID是否为偶数
    bool isEven = (tid % 2 == 0);
    
    // __all_sync: 如果Warp内所有线程的条件都为true，返回1
    int allEven = __all_sync(0xFFFFFFFF, isEven);
    
    // __any_sync: 如果Warp内任意线程的条件为true，返回1
    int anyEven = __any_sync(0xFFFFFFFF, isEven);
    
    // __ballot_sync: 返回一个32位掩码，每个bit对应一个线程的条件值
    unsigned int ballot = __ballot_sync(0xFFFFFFFF, isEven);
    
    // 只让每个Warp的第一个线程打印
    if (laneId == 0) {
        printf("Warp %d: all_even=%d, any_even=%d, ballot=0x%08X\n", 
               warpId, allEven, anyEven, ballot);
    }
}

int main() {
    printf("==========================================\n");
    printf("      Warp级别投票函数演示\n");
    printf("==========================================\n\n");
    
    printf("启动配置: 1个Block, 64个线程 (2个Warp)\n\n");
    
    warpVoteDemo<<<1, 64>>>();
    cudaDeviceSynchronize();
    
    printf("\n解释:\n");
    printf("- ballot = 0x55555555 表示bit 0,2,4,6,...为1 (偶数线程)\n");
    printf("- 二进制: 0101 0101 0101 0101 0101 0101 0101 0101\n");
    
    return 0;
}


## 3. Warp分化演示

当Warp内的线程执行不同分支时，会导致串行执行，降低性能。


In [ ]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>

// 演示Warp分化 (糟糕的写法)
__global__ void warpDivergenceDemo() {
    int tid = threadIdx.x;
    int laneId = tid % 32;
    
    volatile int result = 0;
    
    // Warp分化: 前16个线程和后16个线程执行不同分支
    if (laneId < 16) {
        for (int i = 0; i < 100; i++) result += i;
    } else {
        for (int i = 0; i < 100; i++) result -= i;
    }
}

// 无Warp分化 (好的写法)
__global__ void noDivergenceDemo() {
    int tid = threadIdx.x;
    int warpId = tid / 32;
    
    volatile int result = 0;
    
    // 整个Warp执行同一分支
    if (warpId == 0) {
        for (int i = 0; i < 100; i++) result += i;
    } else {
        for (int i = 0; i < 100; i++) result -= i;
    }
}

int main() {
    printf("==========================================\n");
    printf("        Warp分化性能对比\n");
    printf("==========================================\n\n");
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    float time1, time2;
    
    // 预热
    warpDivergenceDemo<<<1000, 64>>>();
    noDivergenceDemo<<<1000, 64>>>();
    cudaDeviceSynchronize();
    
    // 测试有分化的情况
    cudaEventRecord(start);
    for (int i = 0; i < 1000; i++) {
        warpDivergenceDemo<<<1000, 64>>>();
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&time1, start, stop);
    
    // 测试无分化的情况
    cudaEventRecord(start);
    for (int i = 0; i < 1000; i++) {
        noDivergenceDemo<<<1000, 64>>>();
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&time2, start, stop);
    
    printf("有Warp分化: %.3f ms\n", time1);
    printf("无Warp分化: %.3f ms\n", time2);
    printf("性能提升: %.2fx\n", time1 / time2);
    
    printf("\n结论: 避免Warp内的分支分化可以显著提升性能!\n");
    
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    
    return 0;
}


## 4. 线程与Warp映射关系


In [ ]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void threadWarpMapping() {
    // 计算线程的线性ID
    int tid = threadIdx.x + threadIdx.y * blockDim.x + threadIdx.z * blockDim.x * blockDim.y;
    int warpId = tid / 32;
    int laneId = tid % 32;
    
    // 只打印每个Warp的第一个和最后一个线程
    if (laneId == 0 || laneId == 31) {
        printf("Thread (%d,%d,%d) -> 线性ID=%d, Warp=%d, Lane=%d\n",
               threadIdx.x, threadIdx.y, threadIdx.z,
               tid, warpId, laneId);
    }
}

int main() {
    printf("==========================================\n");
    printf("      线程与Warp映射关系\n");
    printf("==========================================\n\n");
    
    printf("Block配置: (8, 4, 2) = 64线程 = 2个Warp\n\n");
    
    dim3 blockDim(8, 4, 2);  // 8*4*2 = 64 threads
    threadWarpMapping<<<1, blockDim>>>();
    cudaDeviceSynchronize();
    
    printf("\n说明: 线程按照 x->y->z 的顺序线性化后分配到Warp\n");
    
    return 0;
}


## 总结

通过本notebook，你应该理解了：

1. **SM (Streaming Multiprocessor)**
   - GPU的基本计算单元
   - 包含多个CUDA核心、共享内存、寄存器等资源
   
2. **Warp**
   - 32个线程组成的执行单位
   - 同一Warp内的线程执行相同指令
   - Warp分化会导致性能下降
   
3. **关键数值**
   - Warp大小固定为32
   - 每SM的最大线程数、Warp数取决于GPU架构

## 练习

1. 修改代码，使用`__shfl_sync`在Warp内交换数据
2. 编写一个kernel统计每个SM上运行的Block数量
3. 尝试不同的Block大小，观察Occupancy的变化
